PAD

In [12]:
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
# Load CSV into DataFrame
data = pd.read_csv("PAD.csv")


In [13]:
# Display basic statistics
print(data.describe())




               Age  BloodPressure  DiagnosedPAD  CholesterolLevel  \
count  1126.000000    1126.000000   1126.000000       1126.000000   
mean     62.768561     136.847602      0.500000        209.694849   
std      12.796495      23.501873      0.500222         32.116071   
min      40.000000      90.000000      0.000000        150.000000   
25%      52.000000     119.600000      0.000000        183.000000   
50%      63.000000     135.150000      0.500000        210.050000   
75%      73.700000     155.000000      1.000000        235.400000   
max      86.900000     188.900000      1.000000        273.900000   

       PhysicalActivity          BMI  FamilyHistory  BloodSugarLevel  \
count       1126.000000  1126.000000    1126.000000      1126.000000   
mean           3.011577    25.885666       0.507993       126.259575   
std            1.160442     3.846414       0.500158        31.240877   
min            1.000746    18.011153       0.000000        70.176131   
25%            1.9

In [14]:
data.shape

(1126, 13)

PAD

In [15]:
# Check for missing values
print(data.isnull().sum())

# Explore data visually using plots (e.g., using matplotlib or seaborn)

Age                   0
Smoking               0
BloodPressure         0
DiagnosedPAD          0
CholesterolLevel      0
PhysicalActivity      0
BMI                   0
Gender                0
FamilyHistory         0
BloodSugarLevel       0
Diet                  0
StressLevel           0
SedentaryLifestyle    0
dtype: int64


In [16]:
# Check if 'Gender' and 'Diet' columns exist in the DataFrame
if 'Gender' in data.columns and 'Diet' in data.columns:
    # One-hot encode categorical variables
    data = pd.get_dummies(data, columns=["Gender", "Diet"])

    # Encode ordinal variable using a mapping
    ordinal_mapping = {"Low-Fiber": 0, "Balanced": 1, "High-Fat": 2}
    # Make sure to replace 'Diet' with the correct prefix that was created during one-hot encoding
    data["Diet"] = data[["Diet_Balanced", "Diet_High-Fat", "Diet_Low-Fiber"]].idxmax(axis=1).map(ordinal_mapping)
    data.drop(["Diet_Balanced", "Diet_High-Fat", "Diet_Low-Fiber"], axis=1, inplace=True)
else:
    print("Columns 'Gender' and/or 'Diet' not found in the DataFrame.")


In [17]:
from sklearn.preprocessing import StandardScaler

# Extract numerical features
numerical_features = ['BloodPressure', 'CholesterolLevel', 'PhysicalActivity', 'BMI', 'BloodSugarLevel', 'StressLevel']

# Initialize the scaler
scaler = StandardScaler()

# Scale the numerical features
data[numerical_features] = scaler.fit_transform(data[numerical_features])


In [18]:
# Check the available columns in the DataFrame
print(data.columns)

# Identify and handle categorical variables
categorical_columns = ['Smoking', 'Gender', 'Diet']
# Ensure the specified categorical columns exist in the DataFrame
categorical_columns = [col for col in categorical_columns if col in data.columns]

# If categorical columns are found, proceed with one-hot encoding
if categorical_columns:
    data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

    # Features and target variable
    X = data.drop("DiagnosedPAD", axis=1)
    y = data["DiagnosedPAD"]

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize the scaler
    scaler = StandardScaler()

    # Scale the numerical features
    numerical_features = ['BloodPressure', 'CholesterolLevel', 'PhysicalActivity', 'BMI', 'BloodSugarLevel', 'StressLevel']
    X_train[numerical_features] = scaler.fit_transform(X_train[numerical_features])
    X_test[numerical_features] = scaler.transform(X_test[numerical_features])

    # Train a RandomForestClassifier
    model = RandomForestClassifier()
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)


Index(['Age', 'Smoking', 'BloodPressure', 'DiagnosedPAD', 'CholesterolLevel',
       'PhysicalActivity', 'BMI', 'FamilyHistory', 'BloodSugarLevel',
       'StressLevel', 'SedentaryLifestyle', 'Gender_Female', 'Gender_Male',
       'Diet'],
      dtype='object')


In [19]:
data

,Age,BloodPressure,DiagnosedPAD,CholesterolLevel,PhysicalActivity,BMI,FamilyHistory,BloodSugarLevel,StressLevel,SedentaryLifestyle,Gender_Female,Gender_Male,Smoking_Yes
0,54.0,-1.270574,1,0.009506,0.450089,0.293963,0,-0.699973,0.034583,1,True,False,True
1,83.6,-0.674612,1,0.149685,1.349640,-0.505272,0,1.337705,-0.795919,1,False,True,False
2,61.0,-0.844887,0,0.289864,-1.081105,-1.327428,1,-1.255600,0.401359,0,True,False,False
3,72.0,0.730155,1,-1.703793,0.198928,-1.473877,0,-0.300577,-0.836863,0,True,False,True
4,44.0,2.122152,1,2.000048,-0.684187,0.509022,0,1.234403,-0.066125,1,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1121,62.7,-0.078650,1,1.897250,1.355223,0.916011,0,1.049651,1.390404,1,False,True,False
1122,43.0,0.049056,0,-1.579190,-1.419124,-0.253917,1,0.521015,-0.298957,0,True,False,True
1123,68.2,1.934850,0,0.560877,0.995129,-0.175936,1,-0.263733,0.744277,1,False,True,False
1124,64.0,1.198411,0,-0.239701,0.030145,-1.692879,0,-1.439140,0.013898,0,False,True,True
